In [2]:
# import required libraries
from vidgear.gears import CamGear
import cv2
import time

source="https://youtu.be/z545k7Tcb5o"
# Add YouTube Video URL as input source (for e.g https://youtu.be/bvetuLwJIkA)
# and enable Stream Mode (`stream_mode = True`)
stream = CamGear(
    source=source, stream_mode=True, logging=True,  time_delay=0
).start()
video_metadata=stream.ytv_metadata

print(video_metadata.keys())

print(video_metadata['fps'])
print(video_metadata['format'])
print(video_metadata['format_index'])

# search available resolution
resolutions=[format['resolution'] for format in video_metadata['formats']]
for res in resolutions:
    print(res)

# select the desired resolution to get right url 
desired_resolution = '1280x720'
for format in video_metadata['formats']:
    
    if format['resolution'] == desired_resolution:
        VIDEO = format['url']
        break

print(VIDEO)



17:25:06 ::    Helper     :: WARNING  :: GStreamer not found!
17:25:06 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
17:25:06 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
17:25:07 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, best, worst]
17:25:07 ::    CamGear    :: WARNING  :: Livestream URL detected. It is advised to use GStreamer backend(`cv2.CAP_GSTREAMER`) with it.
17:25:07 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
17:25:07 ::    CamGear    ::  DEBUG   :: YouTube source ID: `z545k7Tcb5o`, Title: `Périphérique Nord - Porte de la Pape 2024-03-07 17:25`, Quality: `best`
17:25:07 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!


dict_keys(['id', 'title', 'formats', 'thumbnails', 'thumbnail', 'description', 'channel_id', 'channel_url', 'view_count', 'average_rating', 'age_limit', 'webpage_url', 'categories', 'tags', 'playable_in_embed', 'live_status', 'release_timestamp', '_format_sort_fields', 'automatic_captions', 'subtitles', 'comment_count', 'chapters', 'heatmap', 'concurrent_view_count', 'channel', 'channel_follower_count', 'uploader', 'uploader_id', 'uploader_url', 'upload_date', 'availability', 'original_url', 'webpage_url_basename', 'webpage_url_domain', 'extractor', 'extractor_key', 'playlist', 'playlist_index', 'display_id', 'fulltitle', 'release_date', 'is_live', 'was_live', 'requested_subtitles', '_has_drm', 'epoch', 'format_id', 'format_index', 'url', 'manifest_url', 'tbr', 'ext', 'fps', 'protocol', 'preference', 'quality', 'has_drm', 'width', 'height', 'vcodec', 'acodec', 'dynamic_range', 'source_preference', 'resolution', 'aspect_ratio', 'http_headers', 'video_ext', 'audio_ext', 'vbr', 'abr', 'fo

In [3]:
import numpy as np
import supervision as sv
import cv2
import os
from collections import defaultdict, deque
# os.environ.pop("QT_QPA_PLATFORM_PLUGIN_PATH")
# from supervision import draw_text , Color
from ultralytics import YOLO
# load yolo model and get class name
MODEL = "yolov8s.pt"
model=YOLO(MODEL)
CLASS_NAMES_DICT = model.model.names
print(CLASS_NAMES_DICT)
# load openvino model to get faster FPS 
model = YOLO("/media/storage/OneDrive-4iTEC/Documents/GitHub/My_notebooks/traffic_analysis/yolov8s_openvino_model/", task='detect')
# model=YOLO(MODEL)
# model.fuse()

colors = sv.ColorPalette.LEGACY

video_info = sv.VideoInfo.from_video_path(VIDEO)
print(video_info)
# calculate ratio between video stream and displayed size (here's 1280)

coef=video_info.width/1280
# print(coef)

# polygon design 
#  ----> x
# |         (x4,y4)   (x3,y3)
# |              +-------+
#               +-------+
# y            +-------+
#         (x1,y1)    (x2,y2)

# 3 polygons so 3 values in each coordinate from left to right 
#    [zone1,zone2, zone3]
x1 = [-160 , -25 , 971  ] 
y1 = [ 405 , 710 , 671  ]
x2 = [ 112 , 568 , 1480 ]
y2 = [ 503 , 710 , 671  ]
x3 = [ 557 , 706 , 874  ]
y3 = [ 195 , 212 , 212  ]
x4 = [ 411 , 569 , 749  ]
y4 = [ 195 , 212 , 212  ]
# transform according video stream and displayed video ratio 
x1, y1, x2, y2, x3, y3, x4, y4 = map(lambda x: [value * coef for value in x], [x1, y1, x2, y2, x3, y3, x4, y4])


# search middle point of the polygon (x1+x4)/2) or tier point from top ( x1 + 2* x4) / 3) to draw line for counting 
x14 = [( x1 + 2 * x4) / 3
       for x1,x4
       in zip(x1,x4)]
y14 = [( y1 + 2 * y4) / 3
       for y1,y4
       in zip(y1,y4)]
x23 = [ ( x2 + 2 * x3) / 3
       for x2,x3
       in zip(x2,x3)]
y23 = [( y2 + 2 * y3) / 3
       for y2,y3
       in zip(y2,y3)]

# polygon zone from left to right (becarefull must be in the same order than le linezone)
polygons = [
  np.array([
 [x1, y1],[x2 , y2],[x3 , y3],[x4 , y4]
  ],np.int32)
 for x1,y1,x2,y2,x3,y3,x4,y4
 in zip(x1,y1,x2,y2,x3,y3,x4,y4)
]


# initialize our zones

zones = [
    sv.PolygonZone(
        polygon=polygon,
        frame_resolution_wh=video_info.resolution_wh
    )
    for polygon
    in polygons
]
zone_annotators = [
    sv.PolygonZoneAnnotator(
        zone=zone,
        color=colors.by_idx(index),
        thickness=2,
        text_thickness=1,
        text_scale=0.5,
    )
    for index, zone
    in enumerate(zones)
]

label_annotators=[
    sv.LabelAnnotator(
        text_position=sv.Position.TOP_CENTER,
        color=colors.by_idx(index),
        text_thickness=1,
        text_scale=0.5,
        )
        for index 
        in range(len(zones))
]

# box_annotators = [
#     sv.BoxAnnotator(
#         color=colors.by_idx(index),
#         thickness=1,
#         text_thickness=1,
#         text_scale=0.5
#         )
#     for index
#     in range(len(polygons))
# ]
box_annotators = [
    sv.BoundingBoxAnnotator(
        color=colors.by_idx(index),
        thickness=1,
        )
    for index
    in range(len(polygons))
]

trace_annotators=[
    sv.TraceAnnotator(
        color=colors.by_idx(index),
        thickness=1,
        trace_length=video_info.fps * 1.5,
        position=sv.Position.BOTTOM_CENTER,
        )
    for index
    in range(len(polygons))
]


lines_start=[
   
    sv.Point(x14, y14)
    for x14,y14
    in zip(x14,y14)
 
]

lines_end =[
    
    sv.Point(x23, y23)
    for x23,y23
    in zip(x23,y23)
]

positions=[(sv.Position.CENTER,sv.Position.CENTER),
           (sv.Position.CENTER,sv.Position.CENTER),
           (sv.Position.CENTER,sv.Position.CENTER),
          ]

line_zones=[ sv.LineZone(start=line_start, end=line_end, triggering_anchors=position)
            for line_start, line_end, position
            in zip(lines_start,lines_end,positions)
]

# for automatic line zone annotator not use here want to use a custom one
line_zone_annotators=[sv.LineZoneAnnotator(thickness=1,
                                           color=colors.by_idx(index),
                                            text_thickness=1,
                                              text_scale=0.5,
                                                text_offset=4)
    for index
    in range(len(line_zones))
]

# couting line zone text position 
text_pos=[ sv.Point (x=100,y=320),
            sv.Point (x=700,y=320),
            sv.Point (x=1077,y=320)

]
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=100, match_thresh=0.8, frame_rate=video_info.fps)

# byte_tracker = sv.ByteTrack()
fps_monitor=sv.FPSMonitor()
heat_map = sv.HeatMapAnnotator ()
smoother = sv.DetectionsSmoother()




100%|██████████| 21.5M/21.5M [00:02<00:00, 7.63MB/s]


{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [4]:
SOURCES = np.array([[
    [x4[0], y4[0]], 
    [x3[0], y3[0]], 
    [x2[0], y2[0]], 
    [x1[0], y1[0]]

],[ [x4[1], y4[1]], 
    [x3[1], y3[1]], 
    [x2[1], y2[1]], 
    [x1[1], y1[1]]
],

[
    [x4[2], y4[2]], 
    [x3[2], y3[2]], 
    [x2[2], y2[2]], 
    [x1[2], y1[2]]
]])

#zone1 in meters
TARGET_WIDTH = 6
TARGET_HEIGHT = 75

TARGETS = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

#zone 2 in meters
TARGET_WIDTH = 6
TARGET_HEIGHT = 85

TARGETS= np.append(TARGETS, np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
]), axis=0)

#zone3 in meters
TARGET_WIDTH = 6
TARGET_HEIGHT = 80


TARGETS = np.append(TARGETS, np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
]),axis=0)

TARGETS = TARGETS.reshape(3, 4, 2)



class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points

        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(
                reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

# create the transformers matrix for each zone
view_transformers=[ViewTransformer(source=s, target=t)
                  for s,t
                  in zip(SOURCES, TARGETS)]


In [5]:
labels = []

selected_classes = [2, 3, 5, 7] # car, motorcycle, bus, truck from coco classes
# initialize the dictionary that we will use to store the coordinates for each zone
coordinates = defaultdict(lambda: deque(maxlen=30))
coordinates = np.append(coordinates,defaultdict(lambda: deque(maxlen=30)))
coordinates = np.append(coordinates,defaultdict(lambda: deque(maxlen=30)))                     

def process_frame(frame: np.ndarray, fps) -> np.ndarray:
    speed_labels = [],[],[] 
       
    results = model(frame, imgsz=640, verbose=False)[0]
    # results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[np.isin(detections.class_id, selected_classes)] # filer on selected classes
    detections = byte_tracker.update_with_detections(detections)
    # detections = smoother.update_with_detections(detections)

    # copy frame before annotate                      
    annotated_frame = frame.copy()

    for i, (zone, zone_annotator, box_annotator, trace_annotator, line_zone, line_zone_annotator, label_annotator,line_start, line_end,view_transformer,speed_label,coordinate) in  enumerate(zip(zones, 
                                                                                                                                                       zone_annotators,
                                                                                                                                                       box_annotators,
                                                                                                                                                       trace_annotators,
                                                                                                                                                       line_zones,
                                                                                                                                                       line_zone_annotators,
                                                                                                                                                       label_annotators,
                                                                                                                                                       lines_start,
                                                                                                                                                         lines_end,
                                                                                                                                                         view_transformers,
                                                                                                                                                         speed_labels,
                                                                                                                                                         coordinates)):

        mask = zone.trigger(detections=detections)
        detections_filtered = detections[mask]
      
        points = detections_filtered.get_anchors_coordinates(
                anchor=sv.Position.BOTTOM_CENTER)

        # plug the view transformer into an existing detection pipeline
        
        points = view_transformer.transform_points(points=points).astype(int)
        
        for tracker_id, [_, y] in zip(detections_filtered.tracker_id, points):
            coordinate[tracker_id].append(y)

        # wait to have enough data
        for tracker_id in detections_filtered.tracker_id:
                        if len(coordinate[tracker_id]) < fps/2:
                            # print(coordinates[tracker_id], " - id :", tracker_id, 'len : ', len(coordinates[tracker_id]))
                            speed_label.append(f"#{tracker_id}")
                            
                        else:
                            try:
                                coordinate_start = coordinate[tracker_id][-1]
                                coordinate_end = coordinate[tracker_id][0]
                                distance = abs(coordinate_start - coordinate_end)
                                time = len(coordinate[tracker_id]) / fps
                                speed = distance / time * 3.6
                                speed_label.append(f"{int(speed)} km/h")

                            except: 

                                speed_label.append(f"#{tracker_id}")

                                pass
        # labels = [
        # f"#{tracker_id} "
        # for _,_,_,_,tracker_id in detections_filtered]
        # line_zone.trigger(detections=detections_filtered)
        annotated_frame = sv.draw_line(scene=annotated_frame, start=line_start, end=line_end, color=colors.by_idx(i) )
        # annotated_frame = zone_annotator.annotate(scene=annotated_frame, label=f"Dir. Ouest : {i+random.randint(0,100)}")
        
        annotated_frame = zone_annotator.annotate(scene=annotated_frame, label=f"Dir. Ouest : {line_zone.in_count}") if i==0 else zone_annotator.annotate(scene=annotated_frame, 
                                                                                                                                                          label=f"Dir. Est : {line_zone.out_count}") 
        annotated_frame = label_annotator.annotate(scene=annotated_frame,
                                                  detections=detections_filtered,
                                                  labels=speed_label)
        
        # annotated_frame=line_zone_annotator.annotate(annotated_frame,line_counter=line_zone )
        annotated_frame = box_annotator.annotate(scene=annotated_frame,
                                                  detections=detections_filtered,
                                                  )
        
        annotated_frame = trace_annotator.annotate(scene=annotated_frame,detections=detections_filtered )
        line_zone.trigger(detections=detections_filtered)
        # print(line_zone.in_count)
        # print(line_zone.out_count)
       

    return annotated_frame

In [6]:
# for direct show
cap = cv2.VideoCapture(VIDEO)  
# fps = int(cap.get(cv2.CAP_PROP_FPS))
fps=video_info.fps
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"FPS: {fps}")
print(f"image : {width}x{height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    # frame=cv2.resize(frame,(1280,720))
    show=process_frame(frame,int(fps))
   
    fps_monitor.tick()
    fps = fps_monitor()
    fps_text = f"FPS: {fps:.0f}"
    cv2.putText(show, fps_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    cv2.imshow("Counting", show)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


FPS: 30
image : 1280x720
Loading /media/storage/OneDrive-4iTEC/Documents/GitHub/My_notebooks/traffic_analysis/yolov8s_openvino_model for OpenVINO inference...


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/media/storage/python_envs/supervision/lib/python3.11/site-packages/cv2/qt/plugins"


In [90]:
# to save video instead of displaying it
side=0
output_file = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec vidéo pour le format MP4


out = cv2.VideoWriter(output_file, fourcc, video_info.fps, (video_info.width, video_info.height))

cap = cv2.VideoCapture(VIDEO)  
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"FPS: {fps}")
print(f"image : {width}x{height}")
# Temps de début de l'enregistrement
start_time = time.time()
# Durée de l'enregistrement en secondes
duration = 120

while (time.time() - start_time) < duration:
    ret, frame = cap.read()
    if not ret:
        break
    # frame=cv2.resize(frame,(1280,720))
    show=process_frame(frame,0,fps)
    fps_monitor.tick()
    fps = fps_monitor()
    fps_text = f"FPS: {fps:.0f}"
    cv2.putText(show, fps_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # cv2.imshow("Counting", show)
    out.write(show)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
out.release()
cap.release()
cv2.destroyAllWindows()


FPS: 30
image : 1280x720


In [6]:
from ultralytics import YOLO
# model=YOLO("yolov8m.pt")
# model=YOLO("traffic_analysis.pt")
model=YOLO("yolov8m.pt")
model.export(format='openvino')


Ultralytics YOLOv8.0.201 🚀 Python-3.11.6 torch-2.1.0+cu121 CPU (12th Gen Intel Core(TM) i5-1235U)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients, 78.9 GFLOPs

PyTorch: starting from 'yolov8m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (49.7 MB)

ONNX: starting export with onnx 1.15.0 opset 17...
ONNX: export success ✅ 2.2s, saved as 'yolov8m.onnx' (99.0 MB)

OpenVINO: starting export with openvino 2023.1.0-12185-9e6b00e51cd-releases/2023/1...
OpenVINO: export success ✅ 1.6s, saved as 'yolov8m_openvino_model/' (99.1 MB)

Export complete (5.9s)
Results saved to /media/storage/OneDrive-4iTEC/Documents/GitHub/My_notebooks/traffic_analysis
Predict:         yolo predict task=detect model=yolov8m_openvino_model imgsz=640  
Validate:        yolo val task=detect model=yolov8m_openvino_model imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8m_openvino_model'